In [11]:
from pymongo import MongoClient, ASCENDING
from dotenv import load_dotenv
import os


In [12]:
load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
if not MONGO_URI:
    raise ValueError("❌ MONGO_URI not found in .env")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]

print("✅ Connected to MongoDB Atlas")

✅ Connected to MongoDB Atlas


In [13]:
collections = {
    "faculty": db["Faculty"],
    "students": db["Student"],
    "teams": db["Teams"],
    "admins": db["Admin"],
    "search_history": db["Search_history"],
    "similarity_logs": db["Similarity_logs"]
}

In [14]:
collections["faculty"].create_index([("fac_id", ASCENDING)], unique=True)
collections["students"].create_index([("usn", ASCENDING)], unique=True)
collections["teams"].create_index([("team_id", ASCENDING)], unique=True)
collections["admins"].create_index([("admin_id", ASCENDING)], unique=True)
collections["search_history"].create_index([("srch_id", ASCENDING)], unique=True)
collections["similarity_logs"].create_index([("log_id", ASCENDING)], unique=True)

print("✅ Primary Keys (unique indexes) created successfully")


✅ Primary Keys (unique indexes) created successfully


In [15]:
def validate_fk(collection, key, value):
    """Simulate FK constraint by checking if the reference exists."""
    if value is None:
        return True
    return collection.find_one({key: value}) is not None



In [16]:

def insert_faculty(faculty):
    collections["faculty"].insert_one(faculty)
    print("📌 Faculty inserted.")

def insert_student(student):
    # FK: student.team_id → Teams.team_id
    if not validate_fk(collections["teams"], "team_id", student.get("team_id")):
        raise ValueError("❌ Foreign key error: Invalid team_id in Student")
    collections["students"].insert_one(student)
    print("📌 Student inserted.")


def insert_team(team):
    # FK: team.fac_id → Faculty.fac_id
    if not validate_fk(collections["faculty"], "fac_id", team.get("fac_id")):
        raise ValueError("❌ Foreign key error: Invalid fac_id in Team")
    collections["teams"].insert_one(team)
    print("📌 Team inserted.")


def insert_project(project):
    # FK: project.team_id → Teams.team_id
    if not validate_fk(collections["teams"], "team_id", project.get("team_id")):
        raise ValueError("❌ Invalid team_id (FK constraint failed)")

    # FK: project.fac_id → Faculty.fac_id
    if not validate_fk(collections["faculty"], "fac_id", project.get("fac_id")):
        raise ValueError("❌ Invalid fac_id (FK constraint failed)")

    collections["projects"].insert_one(project)
    print("📌 Project inserted.")


def insert_embedding(embedding):
    # FK: embedding.p_id → Projects.p_id
    if not validate_fk(collections["projects"], "p_id", embedding.get("p_id")):
        raise ValueError("❌ Invalid p_id in Embedding")

    collections["embeddings"].insert_one(embedding)
    print("📌 Embedding inserted.")


def insert_search_history(history):
    # FK: matched project ID exists
    if not validate_fk(collections["projects"], "p_id", history.get("matched_pid")):
        raise ValueError("❌ Invalid matched_pid in Search History")

    collections["search_history"].insert_one(history)
    print("📌 Search history inserted.")


def insert_similarity_log(log):
    # FK: log.p_id must exist
    if not validate_fk(collections["projects"], "p_id", log.get("p_id")):
        raise ValueError("❌ Invalid p_id in Similarity Log")

    # FK: matched_pid must exist
    if not validate_fk(collections["projects"], "p_id", log.get("matched_pid")):
        raise ValueError("❌ Invalid matched_pid in Similarity Log")

    collections["similarity_logs"].insert_one(log)
    print("📌 Similarity log inserted.")

In [17]:
print("🎉 MongoDB schema setup complete with PK + FK validation!")

🎉 MongoDB schema setup complete with PK + FK validation!


In [18]:
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]

projects_col = db["projects"]
emb_col = db["embeddings"]

print("✅ Connected to MongoDB")



✅ Connected to MongoDB


In [19]:
def project_exists(p_id: str):
    return projects_col.find_one({"p_id": p_id}) is not None



In [20]:
def generate_embed_id():
    count = emb_col.count_documents({})
    return f"E{count+1:04d}"


In [21]:
JSON_PATH = "projects-embedding-augmented.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"📦 Loaded {len(data)} records")


📦 Loaded 404 records


In [23]:
inserted_count = 0

for proj in data:

    project_id = proj.get("project_id")

    # Skip records without ID
    if not project_id:
        print("⚠️ Missing project_id → Skipping")
        continue

    # ❌ Skip member versions like auto001_syn1
    if "_syn" in project_id:
        print(f"⏭️ Skipping team member entry: {project_id}")
        continue

    # ----------------------------------------
    # Create PROJECT entry if not exists
    # ----------------------------------------
    if not project_exists(project_id):

        project_doc = {
            "p_id": project_id,
            "title": proj.get("title", ""),
            "objective": proj.get("objective", ""),
            "tech_stack": proj.get("tech_stack", ""),
            "domain": proj.get("domain", ""),
            "year": proj.get("year", ""),
            "team_id": f"team_{project_id}",     # RULE 1
            "fac_id": "F000",                   # RULE 2
            "dept": "ISE"                       # RULE 3
        }

        projects_col.insert_one(project_doc)
        print(f"🆕 Created Project entry for {project_id}")

    # ----------------------------------------
    # Create EMBEDDING ENTRY
    # ----------------------------------------
    embed_id = generate_embed_id()

    emb_doc = {
        "embed_id": embed_id,
        "p_id": project_id,
        "title": proj.get("title", ""),
        "description": proj.get("description", ""),
        "objective": proj.get("objective", ""),
        "tech_stack": proj.get("tech_stack", ""),
        "domain": proj.get("domain", ""),
        "year": proj.get("year", ""),
        "vector_data": proj.get("embedding", [])
    }

    try:
        emb_col.insert_one(emb_doc)
        inserted_count += 1
        print(f"✅ Inserted embedding: {embed_id} for {project_id}")

    except Exception as e:
        print(f"❌ Embedding insert failed for {project_id}: {e}")


# --------------------------------------------
# DONE
# --------------------------------------------
print(f"\n🎉 Completed! Inserted {inserted_count} embeddings successfully.")

🆕 Created Project entry for auto001
✅ Inserted embedding: E0001 for auto001
⏭️ Skipping team member entry: auto001_syn1
⏭️ Skipping team member entry: auto001_syn2
⏭️ Skipping team member entry: auto001_syn3
🆕 Created Project entry for auto002
✅ Inserted embedding: E0002 for auto002
⏭️ Skipping team member entry: auto002_syn1
⏭️ Skipping team member entry: auto002_syn2
⏭️ Skipping team member entry: auto002_syn3
🆕 Created Project entry for auto003
✅ Inserted embedding: E0003 for auto003
⏭️ Skipping team member entry: auto003_syn1
⏭️ Skipping team member entry: auto003_syn2
⏭️ Skipping team member entry: auto003_syn3
🆕 Created Project entry for auto004
✅ Inserted embedding: E0004 for auto004
⏭️ Skipping team member entry: auto004_syn1
⏭️ Skipping team member entry: auto004_syn2
⏭️ Skipping team member entry: auto004_syn3
🆕 Created Project entry for auto005
✅ Inserted embedding: E0005 for auto005
⏭️ Skipping team member entry: auto005_syn1
⏭️ Skipping team member entry: auto005_syn2
⏭️ S

In [27]:
import json
from pymongo import MongoClient
from dotenv import load_dotenv
from openai import OpenAI
import os
import re


In [28]:
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]
teams_col = db["teams"]

openai_client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ Connected to MongoDB (Teams only)")


✅ Connected to MongoDB (Teams only)


In [29]:
def generate_unique_team_name():
    while True:
        prompt = (
            "Generate a unique CSE-themed team name. "
            "1 or 2 words only. No numbers, punctuation, or quotes. "
            "Avoid generic names like Code Crusaders, Code Wizards, Cyber Guardians, Circuit Breakers. "
            "Return ONLY the name."
        )

        response = openai_client.chat.completions.create(
            model="llama3.2",
            messages=[
                {"role": "system", "content": "Return only the team name."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10,
            temperature=1.2
        )

        name = response.choices[0].message.content.strip()
        name = name.replace('"', "").replace("'", "")

        # Check unique name
        exists = teams_col.find_one({"t_name": name})
        if not exists:
            return name



In [30]:
JSON_PATH = "projects-embedding-augmented.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"📦 Loaded {len(data)} entries")


📦 Loaded 404 entries


In [31]:
main_project_ids = set()

for entry in data:
    pid = entry.get("project_id")
    if not pid:
        continue

    main_pid = pid.split("_syn")[0] if "_syn" in pid else pid
    main_project_ids.add(main_pid)

print(f"🧩 Found {len(main_project_ids)} main projects\n")


🧩 Found 51 main projects



In [19]:
for pid in main_project_ids:

    team_id = f"team_{pid}"

    existing_team = teams_col.find_one({"team_id": team_id})

    if not existing_team:
        # fresh team creation
        t_name = generate_unique_team_name()
        teams_col.insert_one({
            "team_id": team_id,
            "t_name": t_name,
            "fac_id": "F000",
            "approved": "NILL"          # ✅ added
        })
        print(f"🏆 Created Team: {team_id} → {t_name}")

    else:
        # refresh team name to guarantee uniqueness
        t_name = generate_unique_team_name()
        teams_col.update_one(
            {"team_id": team_id},
            {
                "$set": {
                    "t_name": t_name,
                    "approved": "NILL"   # ✅ added (safe)
                }
            }
        )
        print(f"♻️ Updated Team: {team_id} → {t_name}")


print("\n🎉 TEAMS INSERTION COMPLETE — No other tables touched.")


NameError: name 'main_project_ids' is not defined

In [38]:
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]
students_col = db["students"]

print("✅ Connected to MongoDB (Students Only)")


✅ Connected to MongoDB (Students Only)


In [39]:
def extract_author(pid: str):
    if "_syn" in pid:
        # auto001_syn2
        parts = pid.split("_syn")
        main_digits = "".join(re.findall(r"\d+", parts[0]))  # 001
        syn_digits = "".join(re.findall(r"\d+", parts[1]))   # 2
        return f"auth{main_digits}_syn{syn_digits}"
    else:
        # auto001
        digits = "".join(re.findall(r"\d+", pid))
        return f"auth{digits}"


In [40]:
def author_to_usn(author: str):
    nums = "".join(re.findall(r"\d+", author))
    return f"1RV23IS{nums}"



In [41]:
JSON_PATH = "projects-embedding-augmented.json"

with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"📦 Loaded {len(data)} entries")


📦 Loaded 404 entries


In [42]:
groups = {}  # auto001 → {auth001, auth001_syn1, ...}

for entry in data:
    pid = entry.get("project_id")
    if not pid:
        continue

    main_pid = pid.split("_syn")[0]
    if main_pid not in groups:
        groups[main_pid] = set()

    author = extract_author(pid)
    groups[main_pid].add(author)


In [8]:
import os
import json
from dotenv import load_dotenv
from pymongo import MongoClient

# ---- LOAD ENV ----
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]

students_col = db["students"]  # <-- confirmed collection

print("✅ Connected to MongoDB Atlas")

# ---- LOAD JSON ----
JSON_PATH = "data/students_5th_sem.json"
with open(JSON_PATH, "r") as f:
    json_students = json.load(f)

# ---- LOAD MONGO STUDENTS (ORDERED) ----
mongo_students = list(
    students_col.find({}, {"_id": 1}).sort("_id", 1)
)

limit = min(len(mongo_students), len(json_students))
print(f"🔁 Updating first {limit} students")

updated = 0

for i in range(limit):
    real = json_students[i]
    mongo_id = mongo_students[i]["_id"]

    students_col.update_one(
        {"_id": mongo_id},
        {
            "$set": {
                "usn": real["usn"],
                "s_name": real["s_name"],
                "s_mail_id": real["s_mail"]
            }
        }
    )

    updated += 1
    print(f"✔ {i+1}/{limit} → {real['usn']}")

print("\n✅ ORDER-WISE UPDATE COMPLETE")
print(f"✔ Updated: {updated}")
print(f"⏭ Skipped JSON entries: {len(json_students) - limit}")


✅ Connected to MongoDB Atlas
🔁 Updating first 204 students
✔ 1/204 → 1RV23IS001
✔ 2/204 → 1RV23IS002
✔ 3/204 → 1RV23IS003
✔ 4/204 → 1RV23IS004
✔ 5/204 → 1RV23IS005
✔ 6/204 → 1RV23IS006
✔ 7/204 → 1RV23IS007
✔ 8/204 → 1RV23IS008
✔ 9/204 → 1RV23IS009
✔ 10/204 → 1RV23IS010
✔ 11/204 → 1RV23IS011
✔ 12/204 → 1RV23IS012
✔ 13/204 → 1RV23IS013
✔ 14/204 → 1RV23IS014
✔ 15/204 → 1RV23IS015
✔ 16/204 → 1RV23IS016
✔ 17/204 → 1RV23IS017
✔ 18/204 → 1RV23IS018
✔ 19/204 → 1RV23IS019
✔ 20/204 → 1RV23IS020
✔ 21/204 → 1RV23IS021
✔ 22/204 → 1RV23IS022
✔ 23/204 → 1RV23IS023
✔ 24/204 → 1RV23IS024
✔ 25/204 → 1RV23IS025
✔ 26/204 → 1RV23IS026
✔ 27/204 → 1RV23IS027
✔ 28/204 → 1RV23IS028
✔ 29/204 → 1RV23IS029
✔ 30/204 → 1RV23IS030
✔ 31/204 → 1RV23IS031
✔ 32/204 → 1RV23IS032
✔ 33/204 → 1RV23IS033
✔ 34/204 → 1RV23IS034
✔ 35/204 → 1RV23IS035
✔ 36/204 → 1RV23IS036
✔ 37/204 → 1RV23IS037
✔ 38/204 → 1RV23IS040
✔ 39/204 → 1RV23IS041
✔ 40/204 → 1RV23IS042
✔ 41/204 → 1RV23IS043
✔ 42/204 → 1RV23IS044
✔ 43/204 → 1RV23IS045
✔ 44

In [2]:
import os
import json
from datetime import datetime
from pymongo import MongoClient
from dotenv import load_dotenv

# -----------------------------------------
# 1) MongoDB connection
# -----------------------------------------
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]
projects_col = db["projects"]
search_history_col = db["Search_history"]

print("✅ Connected to MongoDB")


# -----------------------------------------
# 2) Auto-generate srch_id
# -----------------------------------------
def generate_srch_id():
    count = search_history_col.count_documents({})
    return f"S{count + 1:05d}"   # S00001, S00002, ...


# -----------------------------------------
# 3) Check project exists
# -----------------------------------------
def project_exists(pid):
    return projects_col.find_one({"p_id": pid}) is not None


# -----------------------------------------
# 4) Import all analysis report files
# -----------------------------------------
REPORT_DIR = "analysis_report"
files = sorted(os.listdir(REPORT_DIR))

print(f"📁 Found {len(files)} report files")

STUDENT_USN = "1RV23IS0011"    # <<--- FIXED AS YOU ASKED


for f in files:
    if not f.endswith(".json"):
        continue

    path = os.path.join(REPORT_DIR, f)
    with open(path, "r", encoding="utf-8") as fp:
        report = json.load(fp)

    # -------------------------------
    # FIX: handle list-wrapped files
    # -------------------------------
    if isinstance(report, list):
        if len(report) == 0:
            print(f"⚠️ Empty report file: {f}")
            continue
        report = report[0]

    # -------------------------------
    # Extract fields
    # -------------------------------
    query_text = report.get("query")
    timestamp = report.get("timestamp")
    results = report.get("results", [])

    print(f"\n📌 Processing: {f}")

    for rank, r in enumerate(results, start=1):

        matched_pid = r.get("id")
        score = r.get("sim_whole")

        if not project_exists(matched_pid):
            print(f"⚠️ Skipping {matched_pid} — project not found")
            continue

        entry = {
            "srch_id": generate_srch_id(),
            "student_usn": STUDENT_USN,
            "query": query_text,
            "matched_pid": matched_pid,
            "score": score,
            "rank": rank,
            "timestamp": timestamp
        }

        search_history_col.insert_one(entry)

        print(f"📝 Inserted Search History {entry['srch_id']} → PID={matched_pid}")


✅ Connected to MongoDB
📁 Found 16 report files

📌 Processing: analysis_reports.json
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found

📌 Processing: json_1.json
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found

📌 Processing: json_10.json
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skipping None — project not found
⚠️ Skippin

In [1]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
if not MONGO_URI:
    raise ValueError("❌ MONGO_URI not found in .env")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]
projects = db["projects"]

print("✅ Connected to MongoDB Atlas")

# ------------------------------------------
# Add approved="OK" ONLY to docs missing it
# ------------------------------------------
result = projects.update_many(
    {"approved": {"$exists": False}},   # only missing field
    {"$set": {"approved": "OK"}}        # default value
)

print(f"✅ Updated {result.modified_count} project documents.")


✅ Connected to MongoDB Atlas
✅ Updated 51 project documents.


In [20]:
result = db["teams"].update_many(
    {},                      # all documents
    {"$set": {"approved": "NILL"}}
)

print(f"✔ Updated {result.modified_count} team documents")


✔ Updated 51 team documents


In [1]:
from pymongo import MongoClient, UpdateOne
from dotenv import load_dotenv
import os

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
if not MONGO_URI:
    raise ValueError("❌ MONGO_URI not found in .env")

client = MongoClient(MONGO_URI)
db = client["rag_project_db"]

search_col = db["Search_history"]

print("✅ Connected to MongoDB Atlas")

# allowed USNs (ONLY these will appear)
allowed_usns = [
    "1RV23IS007",
    "1RV23IS010",
    "1RV23IS015",
    "1RV23IS001",
    "1RV23IS020"
]

docs = list(search_col.find({}, {"_id": 1}))
bulk_ops = []

for i, doc in enumerate(docs):
    bulk_ops.append(
        UpdateOne(
            {"_id": doc["_id"]},
            {"$set": {"student_usn": allowed_usns[i % len(allowed_usns)]}}
        )
    )

if bulk_ops:
    result = search_col.bulk_write(bulk_ops)
    print(f"✅ Updated {result.modified_count} Search_history documents")
else:
    print("⚠️ No documents found")


✅ Connected to MongoDB Atlas
✅ Updated 120 Search_history documents
